In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType
from datetime import datetime

In [2]:
spark = SparkSession.builder.appName("DailyCodingProblem-day-11").getOrCreate()

## **Problem Scenario: E-commerce Orders Analysis**

### **Background**

You work as a Data Engineer for an e-commerce company. The raw order data contains details about customers, products, payment methods, prices, quantities, and order statuses.

Your team needs to generate daily metrics and product-level insights to help business analysts track performance, identify top products, and analyze customer behavior.

---

## **Business Objectives**

1. **Daily Sales Metrics**

   * Total orders per day
   * Total and average revenue per day
   * Number of unique customers per day
   * Percentage of cancelled orders

2. **Top Products Analysis**

   * Most sold products by quantity
   * Highest revenue products per day
   * Product category sales distribution

3. **Customer Insights**

   * Average order value per customer
   * Total orders per customer
   * Top paying customers

4. **Payment Method Analysis**

   * Count of orders by payment method
   * Revenue split by payment method

---

## **Sample Input Table (Simplified)**

| order\_id | customer\_id | product\_id | category    | payment\_method | order\_time         | price  | quantity | total\_amount | status    |
| --------- | ------------ | ----------- | ----------- | --------------- | ------------------- | ------ | -------- | ------------- | --------- |
| O125678   | C1           | P5          | Electronics | Credit Card     | 2025-08-20 09:15:00 | 250.00 | 2        | 500.00        | Completed |
| O245789   | C2           | P10         | Clothing    | UPI             | 2025-08-20 09:30:00 | 50.00  | 1        | 50.00         | Completed |
| O356890   | C1           | P2          | Electronics | PayPal          | 2025-08-20 10:00:00 | 100.00 | 3        | 300.00        | Cancelled |
| O445678   | C3           | P7          | Books       | Debit Card      | 2025-08-20 11:00:00 | 75.00  | 1        | 75.00         | Completed |

---
This problem has **real-world complexity** with:

* Missing values in `category` and `payment_method`
* Cancellations impacting revenue
* Daily aggregation needs
* Top product and customer analysis

---

In [3]:
df = spark.read.format("csv") \
    .options(header=True, inferSchema=True) \
    .load("/home/jupyter/work/data/sources/csv/day-11/ecommerce_orders.csv")

In [4]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- status: string (nullable = true)



In [5]:
df = df.withColumn(
    "date",
    F.to_date(F.col("order_time"))
).withColumn(
    "time",
    F.date_format(F.col("order_time"), "HH:mm:ss")
)

## **Expected Output 1 – Daily Metrics Table**

| date       | total\_orders | total\_revenue | avg\_revenue\_per\_order | unique\_customers | cancelled\_pct |
| ---------- | ------------- | -------------- | ------------------------ | ----------------- | -------------- |
| 2025-08-20 | 300           | 45250.00       | 150.83                   | 180               | 8%             |
| 2025-08-21 | 295           | 46300.50       | 156.27                   | 175               | 9%             |

---

In [24]:
daily_metrics_df = (
    df.groupBy("date")
      .agg(
          F.count_distinct("order_id").alias("total_orders"),
          F.sum(
              F.when(F.col("status") == "Completed", F.col("total_amount")).otherwise(0)
          ).alias("total_revenue"),
          F.count_distinct("customer_id").alias("unique_customers"),
          F.sum(F.when(F.col("status") == "Cancelled", 1).otherwise(0)).alias("cancelled_orders")
      )
      .withColumn("avg_revenue_per_order", 
                  F.when(F.col("total_orders") > 0, 
                         F.col("total_revenue") / F.col("total_orders"))
                   .otherwise(0))
      .withColumn("cancelled_pct", 
                  (F.col("cancelled_orders") / F.col("total_orders")) * 100)
)


daily_metrics_df.show()

+----------+------------+------------------+----------------+----------------+---------------------+------------------+
|      date|total_orders|     total_revenue|unique_customers|cancelled_orders|avg_revenue_per_order|     cancelled_pct|
+----------+------------+------------------+----------------+----------------+---------------------+------------------+
|2025-08-20|         300|         211577.96|             163|              31|    705.2598666666667|10.333333333333334|
|2025-08-22|         300|198231.27000000005|             158|              29|    660.7709000000002| 9.666666666666666|
|2025-08-23|         300|226806.84000000005|             167|              22|    756.0228000000002| 7.333333333333333|
|2025-08-21|         300|         198488.13|             164|              28|             661.6271| 9.333333333333334|
|2025-08-24|         300| 221393.3899999999|             154|              38|    737.9779666666664|12.666666666666668|
+----------+------------+---------------

## **Expected Output 2 – Top Products Table**

| date       | product\_id | category    | total\_quantity | total\_revenue |
| ---------- | ----------- | ----------- | --------------- | -------------- |
| 2025-08-20 | P5          | Electronics | 35              | 8750.00        |
| 2025-08-20 | P10         | Clothing    | 28              | 1400.00        |
| 2025-08-20 | P2          | Electronics | 20              | 6000.00        |

---

In [28]:
top_products_df = top_products_df = (
    df.filter(F.col("status") == "Completed")  
      .groupBy("date", "product_id", "category")
      .agg(
          F.sum("quantity").alias("total_quantity"), 
          F.sum("total_amount").alias("total_revenue")
      )
      .orderBy("date", F.col("total_revenue").desc())
)

top_products_df.show()

+----------+----------+-----------+--------------+------------------+
|      date|product_id|   category|total_quantity|     total_revenue|
+----------+----------+-----------+--------------+------------------+
|2025-08-20|       P40|       Home|            18|           6025.22|
|2025-08-20|       P27|Electronics|            12|           4806.57|
|2025-08-20|       P24|      Books|            14|4075.7999999999997|
|2025-08-20|       P40|   Clothing|             9|3932.7999999999997|
|2025-08-20|       P26|       Home|            13|           3856.45|
|2025-08-20|       P31|   Clothing|             9|           3680.52|
|2025-08-20|       P41|Electronics|            11|           3398.16|
|2025-08-20|       P14|      Books|            13|            3365.8|
|2025-08-20|       P23|Electronics|             7|           2929.17|
|2025-08-20|       P30|     Sports|             6|           2862.24|
|2025-08-20|       P43|   Clothing|             7|           2744.65|
|2025-08-20|        


## **Expected Output 3 – Payment Method Analysis**

| date       | payment\_method | order\_count | total\_revenue |
| ---------- | --------------- | ------------ | -------------- |
| 2025-08-20 | Credit Card     | 120          | 25000.00       |
| 2025-08-20 | UPI             | 80           | 10000.00       |
| 2025-08-20 | PayPal          | 50           | 5200.00        |

---


In [29]:
payment_method_df = df.filter(F.col("status") == "Completed").groupBy(
    "date",
    "payment_method",
).agg(
    F.count("order_id").alias("order_count"),
    F.sum("total_amount").alias("total_revenue")
).orderBy("date")


payment_method_df.show()

+----------+--------------+-----------+------------------+
|      date|payment_method|order_count|     total_revenue|
+----------+--------------+-----------+------------------+
|2025-08-20|    Debit Card|         53| 36810.10999999999|
|2025-08-20|          Cash|         37|          27228.96|
|2025-08-20|          NULL|          4|           2725.61|
|2025-08-20|        PayPal|         57| 48285.51999999998|
|2025-08-20|   Credit Card|         62|          46299.59|
|2025-08-20|           UPI|         56| 50228.16999999999|
|2025-08-21|        PayPal|         49|          35229.47|
|2025-08-21|    Debit Card|         58| 46197.82000000002|
|2025-08-21|   Credit Card|         50|          34317.95|
|2025-08-21|          NULL|          8|5949.9400000000005|
|2025-08-21|           UPI|         57|45653.829999999994|
|2025-08-21|          Cash|         50|31139.120000000003|
|2025-08-22|    Debit Card|         55| 33564.27999999999|
|2025-08-22|           UPI|         58| 43555.4600000000